In [2]:
import torch
from tqdm import tqdm, trange
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, AdamW
import transformers
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import random
from sklearn import metrics
import torch.nn as nn
import argparse
import torch.distributed as dist
import torch.utils.data.distributed
import os
import pickle
import numpy as np
from sklearn.metrics import f1_score
from utils_single import (compute_metrics, convert_examples_to_features,
                        output_modes, processors)
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from transformers import (WEIGHTS_NAME, get_linear_schedule_with_warmup, AdamW,
                          RobertaConfig,
                          RobertaForSequenceClassification,
                          RobertaTokenizer)
from CodeBertModel import TextCNNClassifer
import os
# for i in range(58):
#     os.system('python3 run_classifier.py --model_type roberta --model_name_or_path microsoft/codebert-base --task_name codesearch --do_predict --output_dir ../data/codesearch/test/ --data_dir ../data/codesearch/test/ --max_seq_length 512 --per_gpu_train_batch_size 32 --per_gpu_eval_batch_size 32 --learning_rate 1e-5 --num_train_epochs 8 --test_file aspectj_'+str(i)+'.txt  --pred_model_dir ./models/java/checkpoint-best/ --test_result_dir ./results/java/batch_result_'+str(i)+'.txt')
from more_itertools import chunked
def calculate_same_value(labels_sorted, test_p_sorted, start_pos):
    i = start_pos
    num_same = 0
    num_p = 0
    while test_p_sorted[start_pos] == test_p_sorted[i]:
        num_same = num_same + 1
        if labels_sorted[i] ==1 : num_p = num_p + 1
        i = i + 1
        if i == len(labels_sorted ): break
    return num_p, num_same
def eval_mrr(test_p, labels):#在第二维相似度得分，真实标签
    test_p_sorted = test_p
    test_p_index = sorted(range(len(test_p_sorted)), key=lambda k: test_p_sorted[k], reverse=True)  # 降序排序
    test_p_sorted = sorted(test_p, reverse=True)

    labels_sorted = []
    for index in test_p_index:
        labels_sorted.append(labels[index])

    top_num = 10
    top10rank = 0
    for i in range(top_num):
        if (labels_sorted[i] == 1):
            '''
            num_p, num_s = calculate_same_value(labels_sorted, test_p_sorted, i)
            num_r = top_num - i
            if num_p > (num_s-num_r):
                top10rank = 1
                break
            v1 = perm(num_s-num_r, num_p)*perm(num_s-num_p,num_s-num_p)
            v2 = perm(num_s,num_s)
            top10rank = 1-(float)((float)(v1)/(float)(v2))
            if top10rank > 1: top10rank=1
            if top10rank!=top10rank: top10rank=1
            break

    return top10rank

    '''
            top10rank = 1
            break
    num_p, num_s = calculate_same_value(labels_sorted, test_p_sorted, 10)
    if (num_p >= 1): top10rank = 1  # 统计在第十位并列排名相同的文件中，是否含有相关文件

    MRRrank = 0.0
    for i in range(len(labels_sorted)):
        if (labels_sorted[i] == 1):
            MRRrank = MRRrank + float(1 / (i + 1))
            break

    MAPrank = 0.0
    pos_num = 0
    for i in range(len(labels_sorted)):
        if (labels_sorted[i] == 1):
            pos_num = pos_num + 1
            MAPrank = MAPrank + float(pos_num / (i + 1))
    if pos_num==0:
        print('出现不存在pos的例子')
        pos_num=1
    MAPrank = float(MAPrank / pos_num)
#     MRRrank = float(MRRrank / pos_num)

    return top10rank, MRRrank, MAPrank

def simple_accuracy(preds, labels):
#     print(type(preds),type(labels))
    return (preds == labels).mean()


def acc_and_f1(preds, labels):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds)
    return {
        "acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2,
    }


def compute_metrics(preds, labels):
    assert len(preds) == len(labels)

    return acc_and_f1(preds, labels)
#设置种子，为了结果复现
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

In [7]:
class args(object):
    """A single set of features of data."""

    def __init__(self):
        self.model_type = 'roberta'
        self.output_dir='/data/hdj/data/CodeBERT/codesearch/models/java'
        self.test_result_dir='/data/hdj/data/CodeBERT/codesearch/results/java/test_v1_0414.txt'
        self.start_epoch=0
        self.num_train_epochs=1
        self.model_type='roberta'
        self.config_name=''
        self.model_name_or_path=None
        self.task_name='codesearch'
        self.tokenizer_name=''
        self.model_name_or_path='microsoft/codebert-base'
        self.do_lower_case=True
        self.seed=42
        self.gradient_accumulation_steps=1
        self.weight_decay=0.0
        self.max_grad_norm=1.0
        self.learning_rate=1e-6
        self.adam_epsilon=1e-8
        self.warmup_steps=0
        self.max_steps=-1
        self.num_train_epochs=5
# class args(object):
#     """A single set of features of data."""

#     def __init__(self):
#         self.model_type = 'codesearch'
#         self.output_dir='/data/hdj/data/CodeBERT/codesearch/models/java'
#         self.test_result_dir='/data/hdj/data/CodeBERT/codesearch/results/java/test_v1_0414.txt'
#         self.start_epoch=0
#         self.num_train_epochs=1
#         self.model_type='roberta'
args=args()
#带权重的交叉熵
# weights=torch.tensor([0.4,0.6]).cuda()
# loss_fun=CrossEntropyLoss(weight=weights)
#不带权重的交叉熵
loss_fun=CrossEntropyLoss()
#设置种子 复现结果
set_seed(args)

In [5]:
#加载数据并保存在cache里
def load_and_cache_examples(data_dir,train_file,dev_file,test_file, task, tokenizer,max_seq_length, ttype='train'):
    '''
        data_dir:数据目录
        train_file:训练文件
        dev_file:验证文件
        test_file:测试文件
        task:任务名 固定为codesearch
        tokenizer:分词器
        max_seq_length:最大序列长度 convert_examples_to_features使用到了
        ttype:类型 train dev test
        
        return：包装好的数据集 [all_input_ids, all_input_mask, all_segment_ids, all_label_ids]
    '''
    processor = processors[task]()#拿到CodesearchProcessor
    output_mode = output_modes[task]#输出模式 固定为 classification
    # if os.path.exists(cached_features_file):

    label_list = processor.get_labels()#固定为 ["0","1"]

    '''
        example结构：[InputExample(uid 'test-1',report,code,label)]
    '''
    examples=None
    if ttype == 'train':
        examples = processor.get_train_examples(data_dir, train_file)
        cnn_train = pickle.load(open('/data/hdj/cross_project_trans/encode_data/aspectj_parameters_train.in','rb'))
    elif ttype == 'dev':
        examples = processor.get_dev_examples(data_dir, dev_file)
    elif ttype == 'test':
        #如果是test的话，instances就是[每一行是一条代测试的数据]
        examples, instances = processor.get_test_examples(data_dir, test_file)
        cnn_train = pickle.load(open('/data/hdj/cross_project_trans/encode_data/aspectj_parameters_test.in','rb'))
    '''
        example:[[uid,report,code,label],[]...]
        label_list:['0','1']
        max_seq_length:200
        tokenizer:分词器
        output_mode:输出模式 固定为 classification
        cls_token:用来分类的token     '<s>', 0, 
        sep_token:用来分割语句的token '</s>', 2
        
        return：[  [       ( input_ids=input_ids,
                          input_mask=input_mask,
                          segment_ids=segment_ids,
                          label_id=label_id)
                    ]
                ]
                instances：[["0","5222","i am a noy","public class"],[],,]
    '''
#     features = convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, output_mode,
#                                             cls_token=tokenizer.cls_token,
#                                             sep_token=tokenizer.sep_token,
#                                            )
    #
    report_features =convert_examples_to_features(True,examples, label_list, max_seq_length, tokenizer, output_mode,
                                                cls_token_at_end=bool(args.model_type in ['xlnet']),
                                                # xlnet has a cls token at the end
                                                cls_token=tokenizer.cls_token,
                                                sep_token=tokenizer.sep_token,
                                                cls_token_segment_id=2 if args.model_type in ['xlnet'] else 1,
                                                pad_on_left=bool(args.model_type in ['xlnet']),
                                                # pad on the left for xlnet
                                                pad_token_segment_id=4 if args.model_type in ['xlnet'] else 0)
    code_features =convert_examples_to_features(False,examples, label_list, max_seq_length, tokenizer, output_mode,
                                                cls_token_at_end=bool(args.model_type in ['xlnet']),
                                                # xlnet has a cls token at the end
                                                cls_token=tokenizer.cls_token,
                                                sep_token=tokenizer.sep_token,
                                                cls_token_segment_id=2 if args.model_type in ['xlnet'] else 1,
                                                pad_on_left=bool(args.model_type in ['xlnet']),
                                                # pad on the left for xlnet
                                                pad_token_segment_id=4 if args.model_type in ['xlnet'] else 0)

    # Convert to Tensors and build dataset
    #report的相关信息
    all_report_input_ids = torch.tensor([f.input_ids for f in report_features], dtype=torch.long)
    all_report_input_mask = torch.tensor([f.input_mask for f in report_features], dtype=torch.long)
#     all_report_segment_ids = torch.tensor([f.segment_ids for f in report_features], dtype=torch.long)
    #report的相关信息
    #code的相关信息
    all_code_input_ids = torch.tensor([f.input_ids for f in code_features], dtype=torch.long)
    all_code_input_mask = torch.tensor([f.input_mask for f in code_features], dtype=torch.long)
#     all_code_segment_ids = torch.tensor([f.segment_ids for f in code_features], dtype=torch.long)
    #code的相关信息
    
    #CNN的特征
    all_cnn_report_idx,all_code_idx=cnn_train[4],cnn_train[5]

    #CNN的特征
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in report_features], dtype=torch.long)
    
    dataset = TensorDataset(all_report_input_ids, all_report_input_mask, all_code_input_ids,all_code_input_mask,all_cnn_report_idx,all_code_idx,all_label_ids)
    if (ttype == 'test'):
        return dataset, instances
    else:
        return dataset
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

In [6]:
a=torch.tensor([[1,1],[2,2]])
b=torch.tensor([[3,3],[4,4]])
torch.add(a,b)

tensor([[4, 4],
        [6, 6]])

In [5]:
#数据集的配置参数
# data_dir = "/data/hdj/data/CodeBERT/data/codesearch/train_valid/"
#NPCNN的数据处理地址
data_dir='/home/hdj/cross_project_trans/report/aspectj_train/'
test_dir="/data/hdj/data/CodeBERT/data/codesearch/test/aspectj_test/aspectj/" #注意目前测试集的目录是单独的，更换测试数据需要更改
# train_file = "aspectj_train_small.txt"
# dev_file = "aspectj_val_small.txt"
test_file = "swt_0.txt"#目前只使用一个例子来进行测试，后期增加数据量
train_file='train.txt'
dev_file = "val.txt"
task_name = "codesearch"
max_seq_length = 256 #关键参数设置 整个序列的最大长度
#加载数据集
#这里load_and_cache_examples 是没有将数据进行缓存的，和codeBert里的同名函数意义不一样，所以统一按照每次都读取数据这样的模式来处理
train_set = load_and_cache_examples(data_dir,train_file,dev_file,test_file,task_name,tokenizer,max_seq_length,'train')
# val_set = load_and_cache_examples(data_dir,train_file,dev_file,test_file,task_name,tokenizer,max_seq_length,'dev')
# # test_set, instances=load_and_cache_examples(test_dir,train_file,dev_file,test_file,task_name,tokenizer,max_seq_length,'test')
# #将数据进行batch_size设置
batch_size = 16 #关键参数
training_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=False)
# val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=False)
#测试集顺序不应该打乱，不然后期就对不上了 shuffle的作用是在枚举的时候才会改变顺序 
# testing_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=False)

已经写入数据量 0 of 1668
已经写入数据量 0 of 1668


In [6]:
batch_size = 16 #关键参数
training_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=False)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=False)

In [6]:
#目前train val数据集比较不均衡 test数据集比较均衡
print('训练集比例',sum(list(training_loader.dataset.tensors[6])),len(training_loader.dataset.tensors[6]),sum(list(training_loader.dataset.tensors[6]))/len(training_loader.dataset.tensors[6]))
# print('验证集比例',sum(list(val_loader.dataset.tensors[4])),len(val_loader.dataset.tensors[4]),sum(list(val_loader.dataset.tensors[4]))/len(val_loader.dataset.tensors[4]))
# print('测试集比例',sum(list(testing_loader.dataset.tensors[3])),len(testing_loader.dataset.tensors[3]))

训练集比例 tensor(834) 1668 tensor(0.5000)


In [14]:
#模型1的实现
device = 'cuda' if torch.cuda.is_available() else 'cpu'
google_v=False #当为true时，使用RobertClassification模型
num_labels=2
class TCNNConfig(object):
    def __init__(self):
        self.in_channels=1
        self.out_channels=100
        self.embedding_size=300
        self.vocab_size=100
        self.wordNums=40
        self.codeLineNums=100
        self.reportLineNums=308
        self.window_sizes=[3,4,5]
        self.num_class=2
config_hdj=TCNNConfig()
MODEL_CLASSES = {'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)}
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
config = config_class.from_pretrained(args.config_name if args.config_name else args.model_name_or_path,
                                     num_labels=num_labels, finetuning_task=args.task_name)
if google_v:
    model = model_class.from_pretrained(args.model_name_or_path, from_tf=bool('.ckpt' in args.model_name_or_path),
                                    config=config)
    print('使用RobertClassification 模型')
else:
    model = TextCNNClassifer(config_hdj)
    print('使用TextCNNClassifier模型')
_ = model.to(device)


RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 514,
  "min_length": 0,
  "model_type": "roberta",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 12,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 1,
  "pruned_heads": {},
  "repetition_penalty": 1.0,
  "temperature":

In [7]:
print(model.config_hdj.codeLineNums)

100


In [8]:
if google_v or True:
    #Google: 使用的优化器和迭代器
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    #Google
else:
#     hdj: textCNNClassifier 使用的优化器和计划器
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-05, betas=(0.9, 0.999))
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
#     hdj

# 验证和test的代码

In [4]:
#开始验证模型的有效性 把模型预测结果连同数据一起保存起来 然后再去判断top mrr mAP
#流程 1.训练过程中 每个10轮去在val集上验证 比较f1值是否优于之前，是就保存结果到best，2.每次保存上一次验证的结果 3.最后再在test集上进行测试

#check 词表是否与之前model有很大的不同 如果有那么该怎么改善，之前embedding矩阵是否就作废了
def evaluate(args, model,eval_loader,instances,tokenizer, checkpoint=None, prefix="", mode='dev'):
    '''
        args:模型训练时关键参数
        model：待验证的模型
        eval_loader:待验证的数据集 DataLoader 类型的参数
        instances: 若为test模式，则将原始数据输入进来，然后将预测结果一起保存，等待后面计算TOP MAP MRR使用
        tokenizer:分词器
        checkpoint: 检查点
        prefix:前缀
        mode: 模式 分为 dev 和 test（需要保存数据）
        
        return: dict{'acc':,'f1':,'acc_f1'} 主要关注f1的值 越大说明正例的预测越准
    '''
    #待返回的验证结果
    results={}
    eval_loss = 0.0 #验证损失累加
    nb_eval_steps = 0 #验证批次
    preds = None #保存预测的值
    out_label_ids = None #保存真实标签target
    print('开始验证。。。')
    for _, data in enumerate(eval_loader, 0):#start=0 默认就是0
        with torch.no_grad():
            report_ids = data[0].to(device, dtype=torch.long).cuda(non_blocking=True)
            report_mask = data[1].to(device, dtype=torch.long).cuda(non_blocking=True)
            code_ids = data[2].to(device, dtype=torch.long).cuda(non_blocking=True)
            code_mask = data[3].to(device, dtype=torch.long).cuda(non_blocking=True)
#             cnn_report=data[4].to(device, dtype=torch.long).cuda(non_blocking=True)
#             cnn_code=data[5].to(device, dtype=torch.long).cuda(non_blocking=True)
            cnn_code=embedding(data[5])
            cnn_report=embedding(data[4])
#             y_train=data[6].to(device, dtype=torch.long).cuda(non_blocking=True)
    #         print(train_x1_batch.shape,train_x2_batch.shape)
            cnn_code = cnn_code.reshape(cnn_code.shape[0]*cnn_code.shape[1],cnn_code.shape[2],cnn_code.shape[3])
            cnn_code = cnn_code.to(device, dtype=torch.float).cuda(non_blocking=True)
            cnn_report = cnn_report.to(device, dtype=torch.float).cuda(non_blocking=True)
#             token_type_ids=None#token_type_ids = data[2].to(device, dtype=torch.long).cuda(non_blocking=True) 因为模型不支持2维设置 所以不再输入这个参数
            targets = data[6].to(device, dtype=torch.long).cuda(non_blocking=True)
            if google_v:
                pred = model(input_ids=ids, attention_mask=mask,token_type_ids=None, labels=targets)
                loss=pred[0]
            else:
                pred = model(report_ids, report_mask, code_ids,code_mask,cnn_report,cnn_code)#输出是[batch_size,2]
            # print(pred, target)
                loss = loss_fun(pred, targets)
#             pred_choice = pred.max(1)[1]
#             correct = pred_choice.eq(targets).cpu().sum()
#             metrics = compute_metrics(pred_choice.cpu().numpy(), targets.cpu().numpy())
#             print('[',epoch,': ',_,'/',num_batch,'] ',blue('val')," loss :%.4f" % loss.item(),' , accuracy: ',correct.item() / float(batch_size) ,'true nums:',sum(targets.cpu().numpy()),' ration :',sum(targets.cpu().numpy())/len(targets),"acc :",metrics['acc']," f1 :",metrics['f1'])
#             loss_list_val.append(loss.item())  
            
            eval_loss += loss.item()
            if(_%100==0):
                print(_,'/',len(eval_loader))
        nb_eval_steps += 1
        #将preds值和标签进行保存
        if preds is None:
            if google_v:
                preds = pred[1].detach().cpu()
            else:
                preds=pred.detach().cpu()
            out_label_ids = targets.detach().cpu()
        else:
            if google_v:
                preds = np.append(preds, pred[1].detach().cpu().numpy(), axis=0)
            else:    
                preds = np.append(preds, pred.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, targets.detach().cpu().numpy(), axis=0)       
    print('验证结束，开始计算指标')
    #计算指标，并更新结果
 
    #ndarra转换为tensor类型
    preds=torch.tensor(preds)
    out_label_ids=torch.tensor(out_label_ids)
#     print(type(preds),preds.shape,preds)
#     print(type(out_label_ids),out_label_ids.shape,out_label_ids)    
    pred_choice = preds.max(1)[1]#[batch_size,2]
    correct = pred_choice.eq(out_label_ids).cpu().sum() #eq需要tensor类型的数据
    #但是compute_metrics 需要numpy的数据类型 真烦人
    metrics = compute_metrics(pred_choice.numpy(), out_label_ids.numpy())
    results.update(metrics)
    #计算完成
    
    #下面分dev 和 test 模式来完成数据模型的保存工作
    if(mode=='dev'):
        #这里只是把result字典里acc和f1给保存到文件
        output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
        with open(output_eval_file, "a+") as writer:
            print("***** Eval results {} *****".format(prefix))
            writer.write('evaluate %s\n' % checkpoint)
            for key in sorted(results.keys()):
                writer.write("%s = %s\n" % (key, str(results[key])))
    elif(mode=='test'):
        #test结果目录 将test数据连同预测结果进行保存
        if(instances==None or len(instances)==0):
            print('输入的测试数据有问题 为None 或者 长度为0')
        #如果目录不存在就创建
        output_test_file = args.test_result_dir
        output_dir = os.path.dirname(output_test_file)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        #检验创建目录完成
        with open(output_test_file, "w") as writer:
            print("***** Output test results *****")
            preds=F.softmax(preds)
            all_logits = preds.tolist()
            for i, logit in tqdm(enumerate(all_logits), desc='Testing'):#desc是进度条的标题
                #instances即代表test集里每一行数据的值
                instance_rep = '<CODESPLIT>'.join([item.encode('ascii', 'ignore').decode('ascii') for item in instances[i]])
                writer.write(instance_rep + '<CODESPLIT>' + '<CODESPLIT>'.join([str(l) for l in logit]) + '\n')
            #打印验证的结果
            for key in sorted(results.keys()):
                print("%s = %s" % (key, str(results[key])))
    return results

In [23]:
preds=torch.rand(4,2)
preds=F.softmax(preds)
preds

/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


tensor([[0.5357, 0.4643],
        [0.4058, 0.5942],
        [0.4719, 0.5281],
        [0.4896, 0.5104]])

# 训练和验证并保存最好模型

In [10]:
#模型应该保存效果最好的那个，而不是训练到最后的那个模型
print("training!!!")
blue = lambda x: '\033[94m' + x + '\033[0m'
epoch=1
best_f1=0.0 #保存最好的f1的模型
model.zero_grad()
model.train()
set_seed(args)  # Added here for reproductibility (even between python 2 and 3)
#这里就是在指定 一共训练多少Epoch [start_epoch,num_train_epcchs]
t_total = len(training_loader) // args.gradient_accumulation_steps * args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, args.warmup_steps, t_total)

train_iterator = trange(args.start_epoch, int(args.num_train_epochs), desc="Epoch")
w= pickle.load(open('/home/hdj/cross_project_trans/report/aspectj_test_middle/w_parameters.in','rb'))
# embedding = nn.Embedding.from_pretrained(torch.tensor(aspectj_w[0]))#从已有的weight导入embedding
# swt_w= pickle.load(open('/home/hdj/cross_project_trans/report/swt_test_middle/w_parameters.in','rb'))
# w= pickle.load(open('/home/hdj/cross_project_trans/report/zxing_test_middle/w_parameters.in','rb'))
embedding = nn.Embedding.from_pretrained(torch.tensor(w[0]))#从已有的weight导入embedding
for idx, p in enumerate(train_iterator):
    loss_list = []
    acc_list = []
    loss_list_val = []
    acc_list_val = []
    num_batch = len(train_set) / batch_size #总共训练次数
    for _, data in enumerate(training_loader, 0):#start=0 默认就是0
        report_ids = data[0].to(device, dtype=torch.long).cuda(non_blocking=True)
        report_mask = data[1].to(device, dtype=torch.long).cuda(non_blocking=True)
        code_ids = data[2].to(device, dtype=torch.long).cuda(non_blocking=True)
        code_mask = data[3].to(device, dtype=torch.long).cuda(non_blocking=True)
#         cnn_report=data[4].to(device, dtype=torch.long).cuda(non_blocking=True)
#         cnn_code=data[5].to(device, dtype=torch.long).cuda(non_blocking=True)
        
        cnn_code=embedding(data[5])
        cnn_report=embedding(data[4])
        y_train=data[6].to(device, dtype=torch.long).cuda(non_blocking=True)
#         print(train_x1_batch.shape,train_x2_batch.shape)
        cnn_code = cnn_code.reshape(cnn_code.shape[0]*cnn_code.shape[1],cnn_code.shape[2],cnn_code.shape[3])
        cnn_code = cnn_code.to(device, dtype=torch.float).cuda(non_blocking=True)
        cnn_report = cnn_report.to(device, dtype=torch.float).cuda(non_blocking=True)
    #     token_type_ids = data[2].to(device, dtype=torch.long).cuda(non_blocking=True) 因为模型不支持2维设置 所以不再输入这个参数
        targets = data[6].to(device, dtype=torch.long).cuda(non_blocking=True)
        if google_v:
#             outputs = model(ids, mask, token_type_ids)
            outputs = model(input_ids=ids, attention_mask=mask,token_type_ids=None, labels=targets)
            loss=outputs[0]#google loss
#             print(outputs)
#             loss_hdj = loss_fun(outputs[1], targets)
#             print('google loss ',loss,' loss_hdj :',loss_hdj)
            pred_choice = outputs[1].max(1)[1]
        else:
            outputs = model(report_ids, report_mask, code_ids,code_mask,cnn_report,cnn_code)

            # print(outputs)
            loss = loss_fun(outputs, targets)
    #         print('查看target和ouput的类型 检查max函数返回的东西是否正确',type(outputs),type(targets))
        #     print(outputs)
            #取outputs二维中最大维的下标
            pred_choice = outputs.max(1)[1]
        if args.gradient_accumulation_steps > 1:
             loss = loss / args.gradient_accumulation_steps
                
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
        if(_+1)%args.gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            model.zero_grad()
        correct = pred_choice.eq(targets).cpu().sum()
        metrics = compute_metrics(pred_choice.cpu().numpy(), targets.cpu().numpy())
        if _ %40 == 0:
            print('[',epoch,':',_,'/',num_batch,']',"loss :%.4f" % loss.item(),' , accuracy: ',correct.item() / float(batch_size) ,'true nums:',sum(targets.cpu().numpy()),' ration :',sum(targets.cpu().numpy())/len(targets),"acc :",metrics['acc']," f1 :",metrics['f1'])
        loss_list.append(loss.item())
        acc_list.append(correct.item() / float(batch_size))
        
       
    #下面是在训练过程中进行验证 每一轮都会验证一次，保存模型最好的 和最近一次的模型
#     results = evaluate(args, model,val_loader,None ,tokenizer, checkpoint=str(args.start_epoch + idx),mode='dev')
#     print('验证结果 ：',results)
    #torch.save(model, '%s/cls_model_%d.pth' % ('models', epoch))
    #./models/java/checkpoint-last
    last_output_dir = os.path.join(args.output_dir, 'checkpoint-last')
    if not os.path.exists(last_output_dir):
        os.makedirs(last_output_dir)
#     model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
#     model_to_save.save_pretrained(last_output_dir)
    torch.save(model, '%s/cls_model_%d.pth' % (last_output_dir, epoch))
    print("Saving model checkpoint to %s", last_output_dir)
    #保存idx_file 文件 第几轮训练的记录
    idx_file = os.path.join(last_output_dir, 'idx_file.txt')
    with open(idx_file, 'w', encoding='utf-8') as idxf:
        idxf.write(str(args.start_epoch + idx) + '\n')
    #保存idx_file完成
    #保存优化器和计划器
    torch.save(optimizer.state_dict(), os.path.join(last_output_dir, "optimizer.pt"))
    torch.save(scheduler.state_dict(), os.path.join(last_output_dir, "scheduler.pt"))
    print("Saving optimizer and scheduler states to %s", last_output_dir)

#     if (results['f1'] > best_f1):
#         best_f1 = results['f1']
#         #./models/checkpoint-best 检查目录是否存在 否则创建
#         output_dir = os.path.join(args.output_dir, 'checkpoint-best')
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)
#         #保存model
# #         model_to_save = model.module if hasattr(model,'module') else model  # Take care of distributed/parallel training
# #         model_to_save.save_pretrained(output_dir)
#         torch.save(model, '%s/cls_model_%d.pth' % (output_dir, epoch))
# #         torch.save(args, os.path.join(output_dir, 'training_{}.bin'.format(idx)))
#         print("Saving model checkpoint to %s", output_dir)
#         #保存优化器的计划器
#         torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
#         torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
#         print("Saving optimizer and scheduler states to %s", output_dir)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

training!!!


/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


[ 1 : 0 / 104.25 ] loss :0.6985  , accuracy:  0.5 true nums: 8  ration : 0.5 acc : 0.5  f1 : 0.6666666666666666
[ 1 : 40 / 104.25 ] loss :0.6869  , accuracy:  0.5 true nums: 8  ration : 0.5 acc : 0.5  f1 : 0.6666666666666666
[ 1 : 80 / 104.25 ] loss :0.6830  , accuracy:  0.4375 true nums: 7  ration : 0.4375 acc : 0.4375  f1 : 0.6086956521739131
Saving model checkpoint to %s /data/hdj/data/CodeBERT/codesearch/models/java/checkpoint-last


/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
Epoch:  20%|██        | 1/5 [04:13<16:55, 253.98s/it]

...

Saving optimizer and scheduler states to %s /data/hdj/data/CodeBERT/codesearch/models/java/checkpoint-last
[ 1 : 0 / 104.25 ] loss :0.5550  , accuracy:  0.875 true nums: 10  ration : 0.625 acc : 0.875  f1 : 0.9090909090909091
[ 1 : 40 / 104.25 ] loss :0.6260  , accuracy:  0.6875 true nums: 6  ration : 0.375 acc : 0.6875  f1 : 0.6153846153846153
[ 1 : 80 / 104.25 ] loss :0.5190  , accuracy:  0.875 true nums: 9  ration : 0.5625 acc : 0.875  f1 : 0.8888888888888888
Saving model checkpoint to %s /data/hdj/data/CodeBERT/codesearch/models/java/checkpoint-last


Epoch:  80%|████████  | 4/5 [16:57<04:13, 253.97s/it]

Saving optimizer and scheduler states to %s /data/hdj/data/CodeBERT/codesearch/models/java/checkpoint-last
[ 1 : 0 / 104.25 ] loss :0.6372  , accuracy:  0.625 true nums: 6  ration : 0.375 acc : 0.625  f1 : 0.5
[ 1 : 40 / 104.25 ] loss :0.4605  , accuracy:  0.9375 true nums: 10  ration : 0.625 acc : 0.9375  f1 : 0.9523809523809523
[ 1 : 80 / 104.25 ] loss :0.5224  , accuracy:  0.8125 true nums: 11  ration : 0.6875 acc : 0.8125  f1 : 0.8421052631578948
Saving model checkpoint to %s /data/hdj/data/CodeBERT/codesearch/models/java/checkpoint-last


Epoch: 100%|██████████| 5/5 [21:10<00:00, 254.19s/it]

Saving optimizer and scheduler states to %s /data/hdj/data/CodeBERT/codesearch/models/java/checkpoint-last


In [8]:
# results = evaluate(args, model,val_loader,None ,tokenizer, checkpoint=str(args.start_epoch + idx),mode='dev')
#下面开启test模型性能
#加载模型进行测试
# del model
import gc
gc.collect()
# test_dir="/data/hdj/data/CodeBERT/data/codesearch/test/swt_test/swt/"
# test_dir="/data/hdj/data/CodeBERT/data/codesearch/test/aspectj_test/aspectj/" #注意目前测试集的目录是单独的，更换测试数据需要更改
# test_dir='/data/hdj/data/CodeBERT/data/codesearch/test/zxing_test/zxing/'
# test_file = "test_all.txt"#目前只使用一个例子来进行测试，后期增加数据量
# test_dir="/data/hdj/data/CodeBERT/data/codesearch/test/swt_test/swt/" 
test_dir='/home/hdj/cross_project_trans/report/aspectj_test_middle/'
test_file = "all_test.txt"#
task_name = "codesearch"
max_seq_length=256
# train_file = "aspectj_train_small.txt"
# dev_file = "aspectj_val_small.txt"
batch_size=64
test_set, instances=load_and_cache_examples(test_dir,'train_file','dev_file',test_file,task_name,tokenizer,max_seq_length,'test')
testing_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=False)

已经写入数据量 0 of 17400
已经写入数据量 10000 of 17400
已经写入数据量 0 of 17400
已经写入数据量 10000 of 17400


In [ ]:
# testing_loader = DataLoader(dataset=test_set, batch_size=64, shuffle=False, num_workers=4, drop_last=False)

In [15]:
#先不删除模型直接进行预测 之后再删除模型 加载保存下来的模型进行预测
# del model
# import gc
# gc.collect()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
google_v=False #当为true时，使用RobertClassification模型
# num_labels=2
w= pickle.load(open('/home/hdj/cross_project_trans/report/aspectj_test_middle/w_parameters.in','rb'))
# embedding = nn.Embedding.from_pretrained(torch.tensor(aspectj_w[0]))#从已有的weight导入embedding
# swt_w= pickle.load(open('/home/hdj/cross_project_trans/report/swt_test_middle/w_parameters.in','rb'))
# w= pickle.load(open('/home/hdj/cross_project_trans/report/zxing_test_middle/w_parameters.in','rb'))
embedding = nn.Embedding.from_pretrained(torch.tensor(w[0]))#从已有的weight导入embedding
output_dir = os.path.join(args.output_dir, 'checkpoint-best')
model=torch.load('%s/cls_model_%d.pth' % (output_dir, 1))
# model=torch.load('/data/hdj/data/CodeBERT/codesearch/models/java/checkpoint-last/cls_model_1.pth')
test_results = evaluate(args, model,testing_loader,instances ,tokenizer, checkpoint=str(args.start_epoch + 0),mode='test')

开始验证。。。


TypeError: forward() takes 3 positional arguments but 7 were given

In [3]:
preds=torch.tensor([0.3,0.4])
preds = F.softmax(preds)

/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [26]:
preds[1]

tensor(0.0036)

In [30]:
path='/data/hdj/data/CodeBERT/codesearch/results/java/0415_aspectj_summary_code_256_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/0415_aspectj_withou_code_clasMeth_summDesc_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/0415_swt_withou_code_clasMeth_summDesc_result_all.txt'
with open('/data/hdj/data/CodeBERT/codesearch/results/java/0415_aspectj_summary_code_256_result_all.txt','r',encoding='utf-8') as f_in:
    with open('/data/hdj/data/CodeBERT/codesearch/results/java/logit_0415_aspectj_summary_code_256_result_all.txt','w',encoding='utf-8') as f_out:
        for i,line in enumerate(f_in):
            contents=line.split('<CODESPLIT>')
#             print('label :',contents[0],' bug_id :',contents[1],' label_0_score :',contents[5],' label_1_socre :',contents[6])
            preds_logits_before=torch.tensor([float(contents[-2]),float(contents[-1])])
            preds_logits = F.softmax(preds_logits_before)
            contents[-2]=str(float(preds_logits[0]))
            contents[-1]=str(float(preds_logits[1]))
            f_out.write('<CODESPLIT>'.join(contents)+'\n')
#             print('<CODESPLIT>'.join(contents))
#             if i==10:
#                 break

/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


In [15]:
import pandas as pd
data=pd.read_pickle('/home/hdj/cross_project_trans/report/aspectj_test_middle/test0.pkl')
data.shape

(300, 7)

In [11]:
with open('/home/hdj/bug_localization/data/ZXing/ZXing-1.6/com/google/zxing/common/reedsolomon/ReedSolomonDecoder.java','rb')as f:
    lines=f.readlines()
    for i,line in enumerate(lines):
        print(type(str(line)),str(line))
        line=str(line).lstrip().replace('\n', '')
        print(type(str(line)),line)
        if i==10:
            break

<class 'str'> b'com googl zxing common reedsolomon\n'
<class 'str'> b'com googl zxing common reedsolomon\n'
<class 'str'> b'reedsolomondecod reed solomon decod\n'
<class 'str'> b'reedsolomondecod reed solomon decod\n'
<class 'str'> b'gf field\n'
<class 'str'> b'gf field\n'
<class 'str'> b'reedsolomondecod gf field reed solomon decod\n'
<class 'str'> b'reedsolomondecod gf field reed solomon decod\n'
<class 'str'> b'field field\n'
<class 'str'> b'field field\n'
<class 'str'> b'decod receiv two reedsolomonexcept two reed solomon except\n'
<class 'str'> b'decod receiv two reedsolomonexcept two reed solomon except\n'
<class 'str'> b'gfpoli poli gfpoli field receiv gf poli gf poli\n'
<class 'str'> b'gfpoli poli gfpoli field receiv gf poli gf poli\n'
<class 'str'> b'syndromecoeffici two syndrom coeffici two\n'
<class 'str'> b'syndromecoeffici two syndrom coeffici two\n'
<class 'str'> b'datamatrix field equal gf data matrix data matrix field\n'
<class 'str'> b'datamatrix field equal gf data ma

In [2]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

In [3]:
input1=InputExample('123',text_a="i love",text_b=None,label=1)
if(input1.text_a):
    print(input1.text_a)
else:
    print('no data')
if(input1.text_b):
    print(input1.text_b)
else:
    print('no data')

i love
no data


In [32]:
#开始计算TOP MRR MAP指标
#有个问题 如果预测a=[0.2,0.5] b=[0.8,0.7] 这样的话，是将a排在b前 还是相反 ；暂时还是按照分数的绝对大小来判断吧
#每1000验证一次三个指标
# path='/data/hdj/data/CodeBERT/codesearch/results/java/aspectj_withou_code_clasMeth_summDesc_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/aspectj_clasMeth_summDesc_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/aspectj_oversam_60_clasMeth_summDesc_result_all.txt'

# path='/data/hdj/data/CodeBERT/codesearch/results/java/swt_clasMeth_summDesc_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/swt_overoversam_60_clasMeth_summDesc_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/swt_withou_code_clasMeth_summDesc_result_all.txt'

# path='/data/hdj/data/CodeBERT/codesearch/results/java/512_0202_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/result_all.txt'
path='/data/hdj/data/CodeBERT/codesearch/results/java/logit_0415_aspectj_summary_code_256_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/0415_aspectj_withou_code_clasMeth_summDesc_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/0415_swt_withou_code_clasMeth_summDesc_result_all.txt'
with open(path,'r',encoding='utf-8') as f_in:
    data = f_in.readlines()
    print('读取的文件长度 :',len(data))
    #注意swt是479 aspectj是1000 zxing是390
#     batched_data = chunked(data, 479)
#     batched_data = chunked(data, 390)
    batched_data = chunked(data, 1000)
    print("start processing")
    top10ranks=[]
    MRRranks=[]
    MAPranks=[]
    for batch_idx, batch_data in enumerate(batched_data):
        preds=[]
        out_label_ids=[]
        report_ids=[]
        paths=[]
        for d_idx, d in enumerate(batch_data):
            line=d.split('<CODESPLIT>')
            out_label_ids.append(int(line[0]))
            preds.append(float(line[-1]))
            report_ids.append(line[1])
            paths.append(line[2])
        # print(len(preds),len(out_label_ids))
        top10rank, MRRrank, MAPrank = eval_mrr(preds, out_label_ids)
        print(batch_idx,'top10rank, MRRrank, MAPrank',top10rank, MRRrank, MAPrank)
        top10ranks.append(top10rank)
        MRRranks.append(MRRrank)
        MAPranks.append(MAPrank)
        # with open('/data/hdj/data/CodeBERT/data/codesearch/test/aspectj_result_check/aspectj_'+str(batch_idx)+'.txt','w',encoding='utf-8') as f_out:
        #     # f_out.writelines('\n'.join(out_label_ids))
        #      for label,pred,id,path in zip(out_label_ids,preds,report_ids,paths):
        #          f_out.write(str(label)+" "+str(pred)+' '+id+' '+path+'\n')

    print('最后平均得分 top10rank, MRRrank, MAPrank :', sum(top10ranks)/len(top10ranks), sum(MRRranks)/len(MRRranks),sum(MAPranks)/len(MAPranks) )

读取的文件长度 : 119000
start processing
0 top10rank, MRRrank, MAPrank 0 0.007874015748031496 0.00490801795632094
1 top10rank, MRRrank, MAPrank 0 0.0016025641025641025 0.0016025641025641025
2 top10rank, MRRrank, MAPrank 0 0.002136752136752137 0.0031387694431172695
3 top10rank, MRRrank, MAPrank 0 0.003236245954692557 0.003236245954692557
4 top10rank, MRRrank, MAPrank 1 1.0 1.0
5 top10rank, MRRrank, MAPrank 0 0.0017271157167530224 0.0030505366631707206
6 top10rank, MRRrank, MAPrank 0 0.008264462809917356 0.008264462809917356
7 top10rank, MRRrank, MAPrank 0 0.003115264797507788 0.003115264797507788
8 top10rank, MRRrank, MAPrank 0 0.0036231884057971015 0.0036231884057971015
9 top10rank, MRRrank, MAPrank 1 0.25 0.25
10 top10rank, MRRrank, MAPrank 0 0.003048780487804878 0.003048780487804878
11 top10rank, MRRrank, MAPrank 1 1.0 0.5175438596491229
12 top10rank, MRRrank, MAPrank 0 0.002304147465437788 0.002304147465437788
13 top10rank, MRRrank, MAPrank 1 1.0 1.0
14 top10rank, MRRrank, MAPrank 0 0.0120

In [35]:
test_p_sorted=[]
labels=[]
data=[]
with open('/data/hdj/data/CodeBERT/codesearch/results/java/logit_0415_aspectj_summary_code_256_result_all.txt','r',encoding='utf-8') as f_in:
    for i,line in enumerate(f_in,1):
#         if i<=1000:
#             continue
        line=line.split('<CODESPLIT>')
        print(i,line[0],line[1],line[2],line[5],line[6])
        test_p_sorted.append(line[6])
        labels.append(line[0])
        data.append(line)
        if i==1000*(0+1):
            break
test_p_index = sorted(range(len(test_p_sorted)), key=lambda k: test_p_sorted[k], reverse=True)  # 降序排序
test_p_sorted = sorted(test_p_sorted, reverse=True)



1 1 251326 tests/bugs163/pr251326/pkgA/f1a83b5 Listener.java 0.998067319393158 0.0019327373011037707

2 1 251326 tests/bugs163/pr251326/pkgA/f1a83b5 Target.java 0.9978675842285156 0.0021324383560568094

3 1 251326 tests/src/org/aspectj/systemtest/ajc163/f1a83b5 Ajc163Tests.java 0.9956315755844116 0.004368429537862539

4 0 251326 weaver/src/org/aspectj/weaver/patterns/936459c WildTypePattern.java 0.9980548620223999 0.0019451744155958295

5 0 251326 ajde/src/org/aspectj/ajde/ui/f1deb9c StructureViewProperties.java 0.9977025389671326 0.0022975008469074965

6 0 251326 tests/bugs/throwsSignature/8660cc1 ExceptionBugTest.java 0.9971248507499695 0.0028751774225383997

7 0 251326 weaver/testsrc/org/aspectj/weaver/patterns/797b6a6 TypePatternListTestCase.java 0.9981250166893005 0.001875002752058208

8 0 251326 weaver/src/org/aspectj/weaver/patterns/6b2d9ae AnnotationPointcut.java 0.9981110095977783 0.0018889514030888677

9 0 251326 tests/src/org/aspectj/systemtest/incremental/e460b1e Incrementa

521 0 251326 tests/src/org/aspectj/systemtest/ajc151/e874581 Ajc151Tests.java 0.990481436252594 0.009518537670373917

522 0 251326 weaver/src/org/aspectj/weaver/bcel/228fc8d BcelCflowAccessVar.java 0.997930645942688 0.0020693964324891567

523 0 251326 weaver/src/org/aspectj/weaver/e76b370 ResolvedTypeMunger.java 0.9979401230812073 0.002059834310784936

524 0 251326 tests/multiIncremental/pr117209/base/src/551b9ca Broken.java 0.9977065324783325 0.002293444937095046

525 0 251326 tests/src/org/aspectj/systemtest/ajc150/508dbcb Ajc150Tests.java 0.970316469669342 0.029683498665690422

526 0 251326 runtime/testsrc/438e61c RuntimeModuleTests.java 0.997791051864624 0.0022089993581175804

527 0 251326 org.aspectj.ajdt.core/src/org/aspectj/ajdt/internal/core/builder/b59b036 AsmElementFormatter.java 0.9959195852279663 0.004080348182469606

528 0 251326 tests/bugs150/pr99125/p/df46c6f I.java 0.9966427087783813 0.003357334528118372

529 0 251326 tests/java5/ataspectj/ataspectj/619a6ad Bug104212.ja

In [36]:
labels_sorted = []
data_sorted=[]
for index in test_p_index:
    labels_sorted.append(labels[index])
    data_sorted.append(data[index])
for i,line in enumerate(data_sorted,1):
    if line[0]=='1' or i<=5:
        print(i,line[0],line[1],line[2],line[5],line[6])

1 0 251326 tests/src/org/aspectj/systemtest/ajc150/fb01cad GenericsTests.java 0.9156460762023926 0.08435390889644623

2 0 251326 tests/bugs/5795b4a VisiblePrivateInterfaceITDs.java 0.9524065256118774 0.04759351536631584

3 0 251326 tests/src/org/aspectj/systemtest/ajc150/6eb77c5 GenericsTests.java 0.9585820436477661 0.041418008506298065

4 0 251326 tests/src/org/aspectj/systemtest/ajc150/7a1002a Ajc150Tests.java 0.9619295001029968 0.038070470094680786

5 0 251326 tests/src/org/aspectj/systemtest/ajc150/2da9b31 Ajc150Tests.java 0.9683043956756592 0.03169555217027664

127 1 251326 tests/src/org/aspectj/systemtest/ajc163/f1a83b5 Ajc163Tests.java 0.9956315755844116 0.004368429537862539

590 1 251326 tests/bugs163/pr251326/pkgA/f1a83b5 Target.java 0.9978675842285156 0.0021324383560568094

867 1 251326 tests/bugs163/pr251326/pkgA/f1a83b5 Listener.java 0.998067319393158 0.0019327373011037707



In [40]:
with open('/data/hdj/SourceFile/data/sourceFile_aspectj/train_valid/aspectj_test.txt','r',encoding='utf-8') as f_in:
    for i,line in enumerate(f_in,1):
        
        line=line.split('<CODESPLIT>')
        print(i,'<label> :',line[0],'<bug_id> :',line[1],'<fixed_file> :',line[2],'<summary>: ',line[3],'<code> :',line[4])
        if i>10:
            break
Anonymous inner classes declare inside an ITD method will cause IllegalAccessError

1 <label> : 1 <bug_id> : 251326 <fixed_file> : tests/bugs163/pr251326/pkgA/f1a83b5 Listener.java <summary>:  Anonymous inner classes declare inside an ITD method will cause IllegalAccessError <code> : packag pkga public interfac listen public void happen string event

2 <label> : 1 <bug_id> : 251326 <fixed_file> : tests/bugs163/pr251326/pkgA/f1a83b5 Target.java <summary>:  Anonymous inner classes declare inside an ITD method will cause IllegalAccessError <code> : packag pkga public class target public listen listena return new listen public void happen string event system println event public static void main string arg target new target (); listena (). happen simpl "); listenb (). happen infer ");

3 <label> : 1 <bug_id> : 251326 <fixed_file> : tests/src/org/aspectj/systemtest/ajc163/f1a83b5 Ajc163Tests.java <summary>:  Anonymous inner classes declare inside an ITD method will cause IllegalAccessError <code> : packag org aspectj systemtest ajc163 public class ajc163test extend org asp